# Clean Four Seam Fastballs

Make a clean sample of four seam fastballs

In [1]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

from random import randrange
import scipy.stats as sst

# ipython reload capabilities for edits to external code
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import pandas as pd

import glob

In [2]:
from src.ellipse import *

In [3]:
# start by selecting your year of interest
year = '2020'

# todo: add cross-year slices

In [4]:
# pull in the codes for statcast queries

PlayerDict = dict()
f = open('data/playerdict'+year+'.txt')

line = f.readline()

line = f.readline()

while line:
    PlayerDict[line.split(',')[0].strip()] = line.split(',')[1].strip()
    line = f.readline()

f.close()
print(PlayerDict['Jose Berrios'])

621244


In [5]:
year = '2020'
plr_files = glob.glob('data/*'+year+'.csv')
saved_plrs = [d.strip('data/').strip('.csv').strip('_'+year) for d in plr_files]

In [6]:
# version where we read in the saved files
BPDict = {}

BPDict[year] = {}

for pindx,pnumber in enumerate(saved_plrs):

    link = plr_files[pindx]
    BPDict[year][pnumber] = pd.read_csv(link, low_memory=False)

In [7]:
year = '2019'
plr_files = glob.glob('data/*'+year+'.csv')
saved_plrs = [d.strip('data/').strip('.csv').strip('_'+year) for d in plr_files]
BPDict[year] = {}

for pindx,pnumber in enumerate(saved_plrs):

    link = plr_files[pindx]
    BPDict[year][pnumber] = pd.read_csv(link, low_memory=False)

In [8]:

indx = 0
plr = saved_plrs[indx]


# here's everything we have access to now!
BPDict[year][plr].keys()

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [9]:
# these are the translations from categories to numbers

PitchDict = {'CH':0,\
             'CU':1,\
             'EP':2,\
             'FO':3,\
             'FA':4,\
             'KN':5,\
             'KC':6,\
             'SC':7,\
             'SI':8,\
             'SL':9,\
             'FC':10,\
             'FS':11,\
            'FT':12,\
            'FF':13,\
            'PO':14,
            'nan':15,
            'CS':16}


iPitchDict = {0:'CH',\
             1:'CU',\
             2:'EP',\
             3:'FO',\
             4:'FA',\
             5:'KN',\
             6:'KC',\
             7:'SC',\
             8:'SI',\
             9:'SL',\
             10:'FC',\
             11:'FS',\
            12:'FT',\
            13:'FF',\
            14:'PO',\
             15:'nan',\
             16:'CS'}



In [10]:
# put the data in a recognisable format

X = []
ptype = []
pname = []
poutcome = []
ballpark = []

year = '2020'
#for plr in check_players:
for plr in saved_plrs:

    Xtmp = np.array([BPDict[year][plr]['release_pos_x'],BPDict[year][plr]['release_pos_z'],\
                 BPDict[year][plr]['plate_x'],BPDict[year][plr]['plate_z'],\
                 BPDict[year][plr]['vx0'],BPDict[year][plr]['vy0'],BPDict[year][plr]['vz0'],\
                 BPDict[year][plr]['ax'],BPDict[year][plr]['ay'],BPDict[year][plr]['az'],\
             BPDict[year][plr]['release_spin_rate'],BPDict[year][plr]['effective_speed'],\
                    BPDict[year][plr]['launch_speed'],BPDict[year][plr]['launch_angle']]).T
    
    for val in BPDict[year][plr]['pitch_type']:
        if val not in PitchDict.keys(): 
            print(val)
            PitchDict[val] = 17
            
    ptypetmp = np.array([PitchDict[x] for x in BPDict[year][plr]['pitch_type']])
    pnametmp = np.array([plr for x in BPDict[year][plr]['pitch_name']])
    poutcometmp = np.array([x for x in BPDict[year][plr]['events']])
    ballparktmp = np.array([x for x in BPDict[year][plr]['home_team']])



    # if you want to select only right handers...
    try:
        if BPDict[year][plr]['p_throws'][0] == 'R':
            for xx in range(0,Xtmp.shape[0]):
                #if np.all(np.isfinite(Xtmp[xx])):
                    X.append(Xtmp[xx])
                    ptype.append(ptypetmp[xx])
                    pname.append(pnametmp[xx])
                    #print(poutcometmp[xx])
                    poutcome.append(poutcometmp[xx])
                    ballpark.append(ballparktmp[xx])
    except:
        pass
        

PT2020 = np.array(ptype)
X2020 = np.array(X)
N2020 = np.array(pname)
O2020 = np.array(poutcome)
B2020 = np.array(ballpark)
print(X2020.shape)

nan
(175816, 14)


In [11]:
# np.array([BPDict[year][plr]['release_pos_x'],BPDict[year][plr]['release_pos_z'],\
                 #BPDict[year][plr]['plate_x'],BPDict[year][plr]['plate_z'],\
                 #BPDict[year][plr]['vx0'],BPDict[year][plr]['vy0'],BPDict[year][plr]['vz0'],\
                # BPDict[year][plr]['ax'],BPDict[year][plr]['ay'],BPDict[year][plr]['az'],\
            # BPDict[year][plr]['release_spin_rate'],BPDict[year][plr]['effective_speed']]).T

f = open('processed/fourseamers_2020_R.csv','w')
print('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16}'\
                  .format('pitcher','outcome','ballpark','xrelease','zrelease','xplate','zplate','vx0','vy0','vz0','ax','ay','az','spin','velocity','launch_speed','launch_angle'),file=f)

for indx in range(0,O2020.size):
    if (PT2020[indx] == 13) & (X2020[indx,10]>0) & (X2020[indx,11]>0):
        print('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16}'\
              .format(N2020[indx],O2020[indx],B2020[indx],X2020[indx,0],X2020[indx,1],\
                          X2020[indx,2],X2020[indx,3],X2020[indx,4],X2020[indx,5],\
                          X2020[indx,6],X2020[indx,7],X2020[indx,8],X2020[indx,9],\
                          X2020[indx,10],X2020[indx,11],X2020[indx,12],X2020[indx,13]),file=f)

f.close()

## Do the same for 2019.

In [12]:
X = []
ptype = []
pname = []
poutcome = []
ballpark = []


year = '2019'
#for plr in check_players:
for plr in saved_plrs:

    Xtmp = np.array([BPDict[year][plr]['release_pos_x'],BPDict[year][plr]['release_pos_z'],\
                 BPDict[year][plr]['plate_x'],BPDict[year][plr]['plate_z'],\
                 BPDict[year][plr]['vx0'],BPDict[year][plr]['vy0'],BPDict[year][plr]['vz0'],\
                 BPDict[year][plr]['ax'],BPDict[year][plr]['ay'],BPDict[year][plr]['az'],\
             BPDict[year][plr]['release_spin_rate'],BPDict[year][plr]['effective_speed'],\
                    BPDict[year][plr]['launch_speed'],BPDict[year][plr]['launch_angle']]).T
    
    for val in BPDict[year][plr]['pitch_type']:
        if val not in PitchDict.keys(): 
            print(val)
            PitchDict[val] = 17
            
    ptypetmp = np.array([PitchDict[x] for x in BPDict[year][plr]['pitch_type']])
    pnametmp = np.array([plr for x in BPDict[year][plr]['pitch_type']])
    poutcometmp = np.array([x for x in BPDict[year][plr]['events']])
    ballparktmp = np.array([x for x in BPDict[year][plr]['home_team']])




    try:
        if BPDict[year][plr]['p_throws'][0] == 'R':
            for xx in range(0,Xtmp.shape[0]):
                #if np.all(np.isfinite(Xtmp[xx])):
                    X.append(Xtmp[xx])
                    ptype.append(ptypetmp[xx])
                    pname.append(pnametmp[xx])
                    poutcome.append(poutcometmp[xx])
                    ballpark.append(ballparktmp[xx])

    except:
        pass
        

PT2019 = np.array(ptype)
X2019 = np.array(X)
N2019 = np.array(pname)
O2019 = np.array(poutcome)
B2019 = np.array(ballpark)


print(X2019.shape)

(404732, 14)


In [13]:
# np.array([BPDict[year][plr]['release_pos_x'],BPDict[year][plr]['release_pos_z'],\
                 #BPDict[year][plr]['plate_x'],BPDict[year][plr]['plate_z'],\
                 #BPDict[year][plr]['vx0'],BPDict[year][plr]['vy0'],BPDict[year][plr]['vz0'],\
                # BPDict[year][plr]['ax'],BPDict[year][plr]['ay'],BPDict[year][plr]['az'],\
            # BPDict[year][plr]['release_spin_rate'],BPDict[year][plr]['effective_speed']]).T

f = open('processed/fourseamers_2019_R.csv','w')
print('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16}'\
                  .format('pitcher','outcome','ballpark','xrelease','zrelease','xplate','zplate','vx0','vy0','vz0','ax','ay','az','spin','velocity','launch_speed','launch_angle'),file=f)

for indx in range(0,O2019.size):
    if PT2019[indx] == 13:
        print('{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14},{15},{16}'\
                  .format(N2019[indx],O2019[indx],B2019[indx],X2019[indx,0],X2019[indx,1],\
                          X2019[indx,2],X2019[indx,3],X2019[indx,4],X2019[indx,5],\
                          X2019[indx,6],X2019[indx,7],X2019[indx,8],X2019[indx,9],\
                          X2019[indx,10],X2019[indx,11],X2019[indx,12],X2019[indx,13]),file=f)

f.close()